In [3]:
# ! pip install konlpy

In [4]:
# ! git clone https://github.com/seujung/KoBART-summarization.git

In [5]:
cd /content/KoBART-summarization/data

/content/KoBART-summarization/data


In [7]:
import torch
import os 
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm

In [8]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla P100-PCIE-16GB
cuda


In [10]:
df = pd.read_csv('/content/drive/MyDrive/sports_news_KoBART.csv')
df = df[['TITLE', 'CONTENT', 'PUBLISH_DT', 'KoBART']]
df

,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...,2021-07-15,유럽축구연맹 유로 2020 심판위원장 로베르토 로세티가 잉글랜드와 덴마크전에 나온 ...
1,디 마리아 없다 유로X코파 베스트11 이탈리아만 7명,지난달 시작된 유로와 코파 아메리카가 11일을 끝으로 막을 내렸다 이탈리아는 결승전...,2021-07-15,지난달 시작된 시작된 유로와 코파 아메리카가 11일을 끝으로 막을 내렸다
2,슈퍼컴퓨터 예측 맨시티 우승맨유 4위 토트넘은 6위,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다 영국 매체 스포츠 바이...,2021-07-15,지난 시즌 초반 고초를 겪었던 맨체스터 시티가 승점 88을 쌓아 경쟁 팀들을 손쉽게...
3,이재성 완벽한 프로 마인츠서 성공할 것 킬 디렉터의 애정 듬뿍 응원,홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다 이재성은 20...,2021-07-15,홀슈타인 킬 우베 스토버 디렉터가 킬의 에이스 이재성을 향해 응원 메시지를 띄웠다
4,홈킷과 딴판 바르사 팬들 NEW 어웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다 팬들은 만족스럽다는 반응이다 바르...,2021-07-15,바르사는 15일 공식 홈페이지를 통해 20212022시즌에 입을 어웨이 킷을 선보였...
...,...,...,...,...
7325,이제 홈팬 야유 받는 먹튀 선수 차비 감독 조차 그만 해라,FC바르셀로나 팬들에게 우스망 뎀벨레는 밉상이 되어버렸다 바르사는 7일 오전 0시 ...,2022-02-07,7일 오전 0시 15분 스페인 바르셀로나 캄노우에서 열린 아틀레티코 마드리드와 20...
7326,오피셜 성남 만 17세 유스 김지수와 준프로 계약,성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다 김지수는 1...,2022-02-07,성남FC가 만17세 2004년생 2004년생 수비수 김지수와 준프로 계약을 체결했다
7327,오베르마스 아약스서 쫓겨난다여성 동료들에게 부적절한 메시지,레전드 마르크 오베르마스가 아약스에서 쫓겨났다 이유는 굉장히 굴욕적이었다 아약스는 ...,2022-02-07,네덜란드 출신의 레전드 마르크 오베르마스가 아약스에서 쫓겨났다 이유는 굉장히 굴욕적이었다
7328,바르사 차비 감독 트라오레 데뷔전 활약에 깜짝몸이 야수 같아,FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다 바...,2022-02-07,지난겨울 이적시장에서 울버햄프턴을 떠나 바르사로 임대 이적했고 이날 경기에 선발 출...


In [11]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [12]:
from konlpy.tag import Okt
okt = Okt()
for i in tqdm(range(len(df))):
  temp_data = okt.morphs(df["CONTENT"].iloc[i])
  temp_list = []
  
  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["CONTENT"].iloc[i] = " ".join(temp_list)
  
  temp_data = okt.morphs(df["TITLE"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["TITLE"].iloc[i] = " ".join(temp_list)

  temp_data = okt.morphs(df["KoBART"].iloc[i])
  temp_list = []

  for word in temp_data:
    if word in stop_words: continue
    temp_list.append(word)
  
  df["KoBART"].iloc[i] = " ".join(temp_list)

100%|██████████| 7330/7330 [06:34<00:00, 18.58it/s]


In [13]:
df.head()

,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결 오른쪽 다리 접촉 있었잖아,유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...,2021-07-15,유럽 축구 연맹 유로 2020 심판 위원장 로베르토 로세티 잉글랜드 덴마크 전 나온...
1,디 마리아 없다 유로 X 코파 베스트 11 이탈리아 만 7 명,지난달 시작 된 유로 코파 아메리카 11일 끝 막 내렸다 이탈리아 는 결승전 잉글랜...,2021-07-15,지난달 시작 된 시작 된 유로 코파 아메리카 11일 끝 막 내렸다
2,슈퍼컴퓨터 예측 맨시티 우승 맨유 4 위 토트넘 은 6 위,새 시즌 시작 하기도 전 슈퍼컴퓨터 예상 한 순위 나왔다 영국 매체 스포츠 바이블 ...,2021-07-15,지난 시즌 초반 고초 겪었던 맨체스터 시티 승점 88 쌓아 경쟁 팀 손쉽게 따돌리고...
3,이재성 완벽한 프로 마인츠 서 성공할 킬 디렉터 애정 듬뿍 응원,홀슈타인 킬 우베 스토 버 디렉터 이재성 향 해 응원 메시지 띄웠다 이재성 은 20...,2021-07-15,홀슈타인 킬 우베 스토 버 디렉터 킬 에이스 이재성 향 해 응원 메시지 띄웠다
4,홈킷 딴판 바르사 팬 NEW 웨이 셔츠 호평 가장 좋아하는 색,FC 바르셀로나 새 시즌 원정 유니폼 공개 했다 팬 은 만족스럽다는 반응 이다 바르...,2021-07-15,바르사 는 15일 공식 홈페이지 통해 20212022시 즌에 입 웨이 킷 선보였는데...


In [14]:
df = df.rename(columns={'TITLE':'summary', 'CONTENT':'news'})
df.columns

Index(['summary', 'news', 'PUBLISH_DT', 'KoBART'], dtype='object')

In [15]:
# 학습 데이터셋 구축
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2, random_state = seed)

In [16]:
train.to_csv('train.tsv', sep='\t')
test.to_csv('test.tsv', sep='\t')

In [17]:
cd /content/KoBART-summarization

/content/KoBART-summarization


In [19]:
! pip install -r requirements.txt

     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 527 kB 54.5 MB/s 
     |████████████████████████████████| 9.1 MB 18.4 MB/s 
     |████████████████████████████████| 6.5 MB 34.4 MB/s 
     |████████████████████████████████| 596 kB 58.1 MB/s 
     |████████████████████████████████| 67 kB 6.5 MB/s 
     |████████████████████████████████| 895 kB 52.1 MB/s 
     |████████████████████████████████| 134 kB 48.2 MB/s 
     |████████████████████████████████| 829 kB 55.8 MB/s 
     |████████████████████████████████| 397 kB 58.6 MB/s 
     |████████████████████████████████| 952 kB 58.2 MB/s 
     |████████████████████████████████| 4.3 MB 49.4 MB/s 
     |████████████████████████████████| 164 kB 49.7 MB/s 
     |████████████████████████████████| 181 kB 61.3 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
     |████████████████████████████████| 111 kB 62.1 MB/s 
     |████████████████████████████████| 1.1 MB 47.8 MB/s 
     |█████████████

In [20]:
! mkdir logs

In [21]:
! python train.py  --gradient_clip_val 1.0  \
                 --max_epochs 50 \
                 --default_root_dir logs \
                 --gpus 1 \
                 --batch_size 4 \
                 --num_workers 4

Downloading: 100% 4.00/4.00 [00:00<00:00, 3.62kB/s]
Downloading: 100% 111/111 [00:00<00:00, 96.2kB/s]
Downloading: 100% 666k/666k [00:00<00:00, 4.71MB/s]
Downloading: 100% 1.33k/1.33k [00:00<00:00, 1.06MB/s]
INFO:root:Namespace(accelerator=None, accumulate_grad_batches=None, amp_backend='native', amp_level=None, auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=None, checkpoint_path=None, default_root_dir='logs', detect_anomaly=False, deterministic=False, devices=None, enable_checkpointing=True, enable_model_summary=True, enable_progress_bar=True, fast_dev_run=False, flush_logs_every_n_steps=None, gpus=1, gradient_clip_algorithm=None, gradient_clip_val=1.0, ipus=None, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=50, max_len=512, max_steps=-1, max_time=None, 